# Анализ поведения пользователей в мобильном приложении: основная часть.

## Описание проекта

В роли аналитика необходимо по заданию менеджера по продукту произвести анализ поведения пользователей в мобильном приложении - "Ненужные Вещи", аналоге "Авито".

## Цель исследования

Управление вовлеченностью клиентов - адаптация приложения по целевой и смежной аудитории. Это будет возможно только на основе данных о поведении пользователей.<br>
Получить на основе поведения пользователей гипотезы о том, как можно было бы улучшить приложение с точки зрения пользовательского опыта.

## Обзор данных

In [ ]:
#Загрузим необходимые для работы модули и библиотеки
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy import stats as st
import numpy as np
import math as mth

In [ ]:
#Загрузим датасет mobile_dataset.csv, просмотрим основную информацию
data = pd.read_csv('/datasets/mobile_dataset.csv')
data.info()
data.sample(5)

Пропуски в данных отсутствуют, но названия столбцов не соответствуют стандартам. А также наблюдаем, что столбец event.time c неверным типом данных.

In [ ]:
#Загрузим датасет mobile_sources.csv, просмотрим основную информацию
source = pd.read_csv('/datasets/mobile_sources.csv')
source.info()
source.sample(5)

Наблюдается аналогичное отсутствие пропусков, но столбец userId потребуется переименовать.

## Предобработка данных

In [ ]:
#Приведем названия столбцов в обоих датасетах к "змеиному" регистру
data.columns = ['event_time', 'event_name', 'user_id']
source.columns = ['user_id', 'source']

In [ ]:
#Приведем столбец event_time к корректному типу, а также уберем миллисекунды
data['event_time'] = pd.to_datetime(data['event_time'], format ='%Y-%m-%d %H:%M:%S')
data['event_time'] = data['event_time'].dt.round('S')

In [ ]:
#Проверим количество явных дубликатов
data.duplicated().sum()

In [ ]:
#Проверим количество явных дубликатов
source.duplicated().sum()

В датасете с источниками установки приложения явные дубликаты остутствуют, в основсном же датасете их количество составляет не более 1,5% от общего - избавимся от них.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
#Выведем список уникальных значений действий пользователя
data['event_name'].value_counts()

Наблюдаются неявные дубликаты в виде действия 'посмотрел номер телефона'. Также имеются разные действия, связанные с поиском по сайту, которые затруднительно охарактеризовать.

In [ ]:
#Удалим неявные дубликаты, объединив действия в 'contacts_show'
data['event_name'] = data['event_name'].replace('show_contacts','contacts_show')

In [ ]:
#Объединим search_1 - search_7 в одно действие
data['event_name'] = data['event_name'].str.replace(r'search_\d+', 'search', regex=True)

In [ ]:
#Проверим результат
data['event_name'].value_counts()

## Исследовательский анализ данных

In [ ]:
#Уточним отрезок времени исследуемых данных
data['event_time'].max() - data['event_time'].min()

In [ ]:
#А также минимальную и максимальную дату зафиксированных действий в данных
display(data['event_time'].min())
display(data['event_time'].max())

Имеются данные за ~28 дней - с 7 октября 00:00:00 по 3 ноября 23:58:13 2019 года.

In [ ]:
#Количество уникальных пользователей
print('Количество уникальных пользователей:', data['user_id'].nunique())

In [ ]:
#Количество событий
print('Количество событий в датасете:',data['event_name'].count())

In [ ]:
event_count = data['event_name'].value_counts()

In [ ]:
fig = px.bar(event_count,
             y='event_name',
             text='event_name',
             color_discrete_sequence = px.colors.sequential.Aggrnyl
            )
fig.update_layout(title='Распределение событий',
                  xaxis_title='Название события',
                  yaxis_title='Количество событий')
fig.show()

Самым частым событием является tips_show (показ рекомендованных объявлений).

In [ ]:
#Проверим, из каких источников пришли пользователи
source.groupby('source').agg({'user_id':'count'}).sort_values(by='user_id', ascending=False)

Наибольшее количество пользователей пришло в приложение из источника yandex.

## Выделение пользовательских сессий и построение воронок по популярным сценариям

In [ ]:
#Перед процедурой выделения сессий отсортируем датасет по 'user_id' и 'event_time'
data = data.sort_values(['user_id', 'event_time'])

In [ ]:
#Определим тайм-аут между событиями, равный 30 мин
g = (data.groupby('user_id')['event_time'].diff() > pd.Timedelta('30Min')).cumsum()
#Добавим столбец с номером сессии
data['session_id'] = data.groupby(['user_id', g], sort=False).ngroup() + 1
data

За основу взяты тайм-ауты по умолчанию из сервисов, предназначенных для оценки посещаемости веб-сайтов и анализа поведения пользователей - Яндекс.Метрика и GoogleAnalytics.<br>
1) https://yandex.ru/support/metrica/general/glossary.html?ysclid=lt4s60nroe897625040<br>
2) https://support.google.com/analytics/answer/2731565?hl=ru#zippy=%2C%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D0%B5

In [ ]:
#Выявим количество дубликатов
data.duplicated(subset=['session_id', 'event_name']).sum()

In [ ]:
#Удалим повторяющиеся события внутри одной сессии
clear_data = data.drop_duplicates(subset=['session_id', 'event_name'])
#Удалим 'tips_show' для построения воронок по активных действиям пользователей
clear_data = clear_data.drop(clear_data[clear_data['event_name'] == 'tips_show'].index)
clear_data

In [ ]:
#Создадим датафрейм с последовательностью событий, совершенными уникальными пользователями
sessions = clear_data.groupby('session_id')['event_name'].apply(tuple).to_frame().reset_index()
sessions

In [ ]:
sessions['event_name'].value_counts()

In [ ]:
#Выведем сессии, в которые просмотрели контакты
sessions['contacts_show'] = sessions['event_name'].apply(lambda x: ('contacts_show' in x))
display(sessions[sessions['contacts_show']==True].head(10))

In [ ]:
clear_data.query('session_id == 6')

In [ ]:
session_contacts_show = sessions.query('contacts_show == True')
session_contacts_show

In [ ]:
#Посмотрим, какие сценарии встречаются чаще
script = session_contacts_show.groupby('event_name')['session_id'].count().sort_values(ascending=False).reset_index()
script.head(10)

Рассмотрим распространенные сценарии. Выберем для построения воронок следующие сценарии, оканчивающиеся целевым событием: 1) открыл карту - посмотрел контакт и 2) воспользовался поиском - посмотрел фото - посмотрел контакт.

In [ ]:
user_map = (data[data['event_name'] == 'map']['user_id']).unique()
user_map_cnt = len(user_map)

user_contacts_show = (data[(data['event_name'] == 'contacts_show') & (data['user_id'].isin(user_map))]['user_id']).unique()
user_contacts_show_cnt = len(user_contacts_show)

funnel_1 = pd.DataFrame({'event_name' : ['map', 'contacts_show'],
                           'user_count' : [user_map_cnt, user_contacts_show_cnt]})
funnel_1

In [ ]:
fig = go.Figure()
fig.add_trace(go.Funnel(y = funnel_1["event_name"],
    x = funnel_1['user_count'], textposition = "inside", 
    textinfo = "value+percent previous"))
fig.update_layout(title='Воронка "map -> contacts_show"')

От просмотревших карту дошли до просмотра номера телефона 19,8% или 289 пользователей.

In [ ]:
user_search = data[data['event_name'] == 'search']['user_id'].unique()
user_search_cnt = len(user_search)

user_photos_show = (data[(data['event_name'] == 'photos_show') & (data['user_id'].isin(user_search))]['user_id']).unique()
user_photos_show_cnt = len(user_photos_show)

user_contacts_show_2 = (data[(data['event_name'] == 'contacts_show') & (data['user_id'].isin(user_photos_show))]['user_id']).unique()
user_contacts_show_2_cnt = len(user_contacts_show_2)

funnel_2 = pd.DataFrame({'event_name' : ['search', 'photos_show', 'contacts_show'],
                           'user_count' : [user_search_cnt, user_photos_show_cnt, user_contacts_show_2_cnt]})
funnel_2

In [ ]:
fig = go.Figure()
fig.add_trace(go.Funnel(y = funnel_2["event_name"],
    x = funnel_2['user_count'],textposition = "inside", 
    textinfo = "value+percent previous"))
fig.update_layout(title='Воронка "search -> photos_show -> contacts_show"')

После поиска фото просмотрели 647 пользователей, что является 38,8% от изначального количества. Что же касается целевого события, то завершили действия данного сценария 192 человека - 11,5% от изначального количества.

## Расчет относительной частоты событий в разрезе двух групп пользователей

In [ ]:
#Запишем общее количество уникальных пользователей в новую переменную
user_total = data['user_id'].unique()

In [ ]:
#Проверим количество пользователей, совершивших целевое событие - просмотр контактов
user_contacts_show = data.query('event_name=="contacts_show"')['user_id'].unique().tolist()
print("Количество пользователей, просмотревших контакты =", len(user_contacts_show))

In [ ]:
#Проверим количество пользователей, не совершавших целевое событие - просмотр контактов
user_no_contacts_show = list( set(user_total) - set(user_contacts_show) )
print("Количество пользователей, не просмотревших контакты =", len(user_no_contacts_show))

In [ ]:
#Количество действий, совершаемое пользователями, просмотревших контакты
yes_contacts_show = data.query('user_id==@user_contacts_show')['event_name'].value_counts().to_frame()
#Для более корректного сравнения количества дейтсвий исключим из первой таблицы просмотры контактов и сопровождающие их звонки по номеру
yes_contacts_show = yes_contacts_show.drop(['contacts_call','contacts_show'])
yes_contacts_show

In [ ]:
#Количество действий, совершаемое пользователями, не просмотревших контакты
no_contacts_show = data.query('user_id==@user_no_contacts_show')['event_name'].value_counts().to_frame()
no_contacts_show

In [ ]:
#Добавим столбец с долей совершаемых действий
yes_contacts_show['percent'] = (yes_contacts_show['event_name']/yes_contacts_show['event_name'].sum()*100).round(1)
no_contacts_show['percent'] = (no_contacts_show['event_name']/no_contacts_show['event_name'].sum()*100).round(1)

In [ ]:
fig = px.bar(yes_contacts_show,
             y='event_name',
             text='percent',
             color_discrete_sequence = px.colors.sequential.Aggrnyl
            )
fig.update_layout(title='Доля и количество действий, совершаемых пользователями, просмотревших контакты',
                  xaxis_title='Название события',
                  yaxis_title='Количество событий')
fig.show()

In [ ]:
fig = px.bar(no_contacts_show,
             y='event_name',
             text='percent',
             color_discrete_sequence = px.colors.sequential.Aggrnyl
            )
fig.update_layout(title='Доля и количество действий, совершаемых пользователями, не просмотревших контакты',
                  xaxis_title='Название события',
                  yaxis_title='Количество событий')
fig.show()

Наибольшее различие связано с действием 'просмотр фото объявления' - в группе, просмотревших контакты, этот показатель на 3,5% больше.
В остальном, разницы относительной частоты событий в разрезе двух групп пользователей - пользователей, которые смотрели/не смотрели контакты, практически не обнаружено. 

## Проверка гипотез

### **Одни пользователи совершают действия tips_show и tips_click , другие - только tips_show.**<br>
Н0: Конверсия в просмотры контактов не
различается у этих двух групп.<br>
Н1: Конверсия в просмотры контактов у этих двух групп различается.

In [ ]:
#Распределим пользователей по группам
tips_show_total = data.query('event_name =="tips_show"')['user_id'].unique().tolist()
print('Общее количество уникальных пользователей, совершивших действие tips_show:', len(tips_show_total))
h_tips_show_click = data.query('event_name =="tips_click" and user_id==@tips_show_total')['user_id'].unique().tolist()
print('Количество уникальных пользователей, совершивших действия tips_show и tips_click:', len(h_tips_show_click))
h_tips_show = list(set(tips_show_total) - set(h_tips_show_click))
print('Количество уникальных пользователей, совершивших только действие tips_show:', len(h_tips_show))

In [ ]:
tips_show_goal = data.query('user_id in @h_tips_show and event_name == "contacts_show"')['user_id'].unique()
print('Количество уникальных пользователей, совершивших только действие tips_show и contacts_show:', len(tips_show_goal))

In [ ]:
tips_show_click_goal = data.query('user_id in @h_tips_show_click and event_name == "contacts_show"')['user_id'].unique()
print('Количество уникальных пользователей, совершивших действия tips_show, tips_click и contacts_show:',
      len(tips_show_click_goal))

In [ ]:
#Проверка гипотезы о равенстве конверсий двух групп 
alpha = 0.05  #Критический уровень статистической значимости

successes = np.array([len(tips_show_goal),len(tips_show_click_goal)])
trials = np.array([len(h_tips_show),len(h_tips_show_click)])

#Пропорция успехов в первой группе
p1 = successes[0]/trials[0]

#Пропорция успехов во второй группе
p2 = successes[1]/trials[1]

#Пропорция успехов в комбинированном датасете
p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

#Разница пропорций в датасетах
difference = p1 - p2 

#Посчитаем статистику в ст.отклонениях стандартного нормального распределения
z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / trials[0] + 1 / trials[1]))

#Зададим стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1)  

p_value = (1 - distr.cdf(abs(z_value))) * 2

print('p-value: ', p_value)
print('Конверсии:',p1, 'и', p2)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print(
        'Не удалось отвергнуть нулевую гипотезу'
    )

Конверсии в просмотры контактов между пользователями, которые совершили события tips_show и tips_click и теми, кто совершил только tips_show, имеют статистически значимое различие.

### **Конверсия в целевое событие между группами, пришедших из источников google и yandex.**<br>
Н0: Конверсия в просмотры контактов не
различается у этих двух групп.<br>
Н1: Конверсия в просмотры контактов у этих двух групп различается.<br>

In [ ]:
#Выделим группы
yandex = source.query('source =="yandex"')['user_id'].unique().tolist()
print('Количество уникальных пользователей, пришедших из yandex:', len(yandex))
google = source.query('source =="google"')['user_id'].unique().tolist()
print('Количество уникальных пользователей, пришедших из google:', len(google))

In [ ]:
#Выявим количество пользователей из yandex, которые совершали целевое действие - просмотр контактов
yandex_contacts = data.query('user_id in @yandex and event_name == "contacts_show"')['user_id'].unique()
print('Количество пользователей из yandex, просмотревших контакты:', len(yandex_contacts))

In [ ]:
#Выявим количество пользователей из google, которые совершали целевое действие - просмотр контактов
google_contacts = data.query('user_id in @google and event_name == "contacts_show"')['user_id'].unique()
print('Количество пользователей из google, просмотревших контакты:', len(google_contacts))

In [ ]:
#Проверка гипотезы о равенстве конверсий двух групп
alpha = 0.05  #Критический уровень статистической значимости

successes = np.array([len(yandex_contacts), len(google_contacts)])
trials = np.array([len(yandex),len(google)])

#Пропорция успехов в первой группе:
p1 = successes[0]/trials[0]

#Пропорция успехов во второй группе:
p2 = successes[1]/trials[1]

#Пропорция успехов в комбинированном датасете:
p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

#Разница пропорций в датасетах
difference = p1 - p2 

#Посчитаем статистику в ст.отклонениях стандартного нормального распределения
z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / trials[0] + 1 / trials[1]))

#Зададим стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1)  

p_value = (1 - distr.cdf(abs(z_value))) * 2

print('p-value: ', p_value)
print('Конверсии:',p1, 'и', p2)

if p_value < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print(
        'Не удалось отвергнуть нулевую гипотезу'
    )

Не выявлено статистически значимого различия между конверсиями пользователей из источников yandex и google.

## Выводы и рекомендации

**В ходе анализа поведения пользователей приложения "Ненужные вещи" загружены необходимые данные, произведена предобработка и исследовательский анализ данных.
Выявлено, что имеются данные за ~28 дней - с 7 октября по 3 ноября 2019 года, количество уникальных пользователей - 4293, а количество событий в датасете - 73079.<br>
Самым частым событием является tips_show (показ рекомендованных объявлений). Наибольшее количество пользователей пришло в приложение из источника yandex.<br>
Был определен тайм-аут между событиями, равный 30 мин и рассмотрены самые распространенные сценарии. Выбраны для построения воронок сценарии, оканчивающиеся целевым событием.
В сценарии №1 "открыл карту - посмотрел контакт" было обнаружено: От просмотревших карту дошли до просмотра номера телефона 19,8% или 289 пользователей.<br>
В сценарии №2 "воспользовался поиском - посмотрел фото - посмотрел контакт" установлено, что после поиска фото просмотрели 647 пользователей, что является 38,8% от изначального количества. Что же касается целевого события, то завершили действия данного сценария 192 человека - 11,5% от изначального количества.<br>
Выявлено, что разницы относительной частоты событий в разрезе двух групп пользователей - пользователей, которые смотрели/не смотрели контакты, практически не обнаружено.
Наибольшее различие связано с действием 'просмотр фото объявления' - в группе, просмотревших контакты, этот показатель на 3,5% больше.<br>
Были проверены 2 гипотезы:<br>
В конверсиях группы, совершившей действия tips_show и tips_click, и группы, совершившей только tips_show - есть статистически значимая разница.
В конверсиях групп пользователей, пришедших из разных источников (yandex и google), статистической разницы конверсии в просмотры контактов обнаружено не было.<br>
<br>
Требуется уделить особое внимание функциям поиска в целях повышении конверсии в целевое событие.
Также, возможно, потребуется пересмотреть алгоритм подбора рекомендованных объявлений для различных пользователей.
Очевидно, что наличие фото в объявлениях увеличивает вероятность того, что пользователь дойдёт до просмотра контактов, стоит подумать над исключением возможности проходить модерацию объявлениям без фотографий.**